### Install requirement

Reference: https://github.com/NVlabs/stylegan2-ada-pytorch/issues/2

In [1]:
%pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 9.5 MB/s eta 0:00:00


#Mounting Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Clone StyleGAN
###Only do once

In [3]:
%cd /content/drive/MyDrive

%rm -rf stylegan2-ada-pytorch/
!git clone https://github.com/woctezuma/stylegan2-ada-pytorch.git

%cd stylegan2-ada-pytorch/
!git checkout google-colab

%cd /content


/content/drive/MyDrive
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 182, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 182 (delta 18), reused 16 (delta 16), pack-reused 159 (from 3)
Receiving objects: 100% (182/182), 1.14 MiB | 2.52 MiB/s, done.
Resolving deltas: 100% (90/90), done.
/content/drive/MyDrive/stylegan2-ada-pytorch
M	.gitignore
M	Dockerfile
M	LICENSE.txt
M	README.md
M	calc_metrics.py
M	dataset_tool.py
M	dnnlib/__init__.py
M	dnnlib/util.py
M	docker_run.sh
M	docs/dataset-tool-help.txt
M	docs/license.html
M	docs/stylegan2-ada-teaser-1024x252.png
M	docs/stylegan2-ada-training-curves.png
M	docs/train-help.txt
M	generate.py
M	legacy.py
M	metrics/__init__.py
M	metrics/frechet_inception_distance.py
M	metrics/inception_score.py
M	metrics/kernel_inception_distance.py
M	metrics/metric_main.py
M	metrics/metric_utils.py
M	metrics/perceptual_path_length.py
M	metrics/precision_recall.py
M	projector.

#Download Pokemon Dataset

In [4]:
!gdown 1RvZbpemMp0Q0U7Y6Z7RphHUPdrd4gcEl


Downloading...
From (original): https://drive.google.com/uc?id=1RvZbpemMp0Q0U7Y6Z7RphHUPdrd4gcEl
From (redirected): https://drive.google.com/uc?id=1RvZbpemMp0Q0U7Y6Z7RphHUPdrd4gcEl&confirm=t&uuid=195f1189-b328-4368-b9e1-de2510748f1c
To: /content/pokemon.zip
100% 41.4M/41.4M [00:00<00:00, 105MB/s]


##Augmentation

In [5]:
# @title Default title text
# Define dataset paths
dataset_zip_path = "/content/pokemon.zip"
extract_folder = "/content/dataset"

import os
import cv2
import numpy as np
from tqdm import tqdm
import albumentations as A
import zipfile  # Import zipfile module

# Extract the dataset if it hasn't been already
if not os.path.exists(os.path.join(extract_folder, "pokemon")):
    with zipfile.ZipFile(dataset_zip_path, 'r') as zip_ref:
        print(f"Extracting {dataset_zip_path} to {extract_folder}...")
        zip_ref.extractall(extract_folder)
    print("✅ Extraction complete!")
else:
    print("✅ Dataset already extracted.")

# Set dataset paths
dataset_path = "/content/dataset"
input_folder = "/content/dataset/pokemon"  # Update based on actual structure
output_folder = "/content/dataset/augmented_images"
os.makedirs(output_folder, exist_ok=True)

# Verify if images exist
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

if not image_files:
    raise ValueError(f"❌ No images found in {input_folder}. Check folder structure.")
else:
    print(f"✅ Found {len(image_files)} images in {input_folder}")

# ... (rest of the code remains the same)

# Define augmentation pipeline
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    # A.RandomBrightnessContrast(p=0.3),
    # A.Blur(blur_limit=3, p=0.2),
    A.Rotate(limit=30, p=0.5),
    # A.GaussNoise(var_limit=50.0, p=0.3),  # Fixed argument
    # A.RandomCrop(height=64, width=64, p=1.0)
])

# Determine number of copies needed (prevent ZeroDivisionError)
target_size = 10000
num_copies = (target_size // max(len(image_files), 1)) + 1

# Apply augmentations & save
counter = 0
for img_file in tqdm(image_files * num_copies):
    img = cv2.imread(os.path.join(input_folder, img_file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    augmented = augmentations(image=img)["image"]
    save_path = os.path.join(output_folder, f"aug_{counter}.png")
    cv2.imwrite(save_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))
    counter += 1
    if counter >= target_size:
        break  # Stop once we have enough images

print(f"✅ Dataset augmented to {counter} images!")

Extracting /content/pokemon.zip to /content/dataset...
✅ Extraction complete!
✅ Found 819 images in /content/dataset/pokemon


 94%|█████████▍| 9999/10647 [01:01<00:03, 162.94it/s]

✅ Dataset augmented to 10000 images!


### Prepare datasets


In [6]:
!python /content/drive/MyDrive/stylegan2-ada-pytorch/dataset_tool.py --source=/content/dataset/augmented_images --dest=/content/dataset/augmented_tfrecords --width=64 --height=64

100% 10000/10000 [00:33<00:00, 297.41it/s]


##Training

In [7]:

augmented_images_path = "/content/dataset/augmented_images"
training_folder = "/content/drive/MyDrive/stylegan2-ada-pytorch/training-runs"

## For first time

In [ ]:
!python /content/drive/MyDrive/stylegan2-ada-pytorch/train.py --outdir={training_folder} --data /content/dataset/augmented_tfrecords --gpus=1 --batch=32 --kimg=5000 --mirror=1 --snap=10 --metrics=none  --cfg_map=8 \
 --augpipe=bg \
 --freezed=10 \
 --resume=ffhq256


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/dataset/augmented_tfrecords",
    "use_labels": false,
    "max_size": 10000,
    "xflip": true,
    "resolution": 64
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {
      "freeze_layers": 10
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,


In [ ]:
!python /content/drive/MyDrive/stylegan2-ada-pytorch/train.py --outdir={training_folder} --data /content/dataset/augmented_tfrecords --gpus=1 --batch=32 --kimg=5000 --mirror=1 --snap=10 --metrics=none  --cfg_map=8 \
 --augpipe=bg \
 --freezed=10 \
 --resume=latest


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/dataset/augmented_images",
    "use_labels": false,
    "max_size": 10000,
    "xflip": true,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 8
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {
      "freeze_layers": 10
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
  